# Reading in the files

In [15]:
import cv2
import numpy as np

#Read the image, convert it into grayscale, and make in binary image for threshold value of 1.
img = cv2.imread('frame.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
_,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)

#Now find contours in it. There will be only one object, so find bounding rectangle for it.

contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]
x,y,w,h = cv2.boundingRect(cnt)

#Now crop the image, and save it into another file.

crop = img[y:y+h,x:x+w]
cv2.imwrite('framemod.png',crop)

True

In [26]:
import numpy as np

def autocrop(image, threshold=0):
    """Crops any edges below or equal to threshold

    Crops blank image to 1x1.

    Returns cropped image.

    """
    if len(image.shape) == 3:
        flatImage = np.max(image, 2)
    else:
        flatImage = image
    assert len(flatImage.shape) == 2

    rows = np.where(np.max(flatImage, 0) > threshold)[0]
    if rows.size:
        cols = np.where(np.max(flatImage, 1) > threshold)[0]
        image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
    else:
        image = image[:1, :1]

    return image

img = cv2.imread('frame.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
crop = autocrop(gray)
cv2.imwrite('framemodOther.png',crop)



True

In [23]:
def crop_image(img,tol=0):
    # img is image data
    # tol  is tolerance
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

img = cv2.imread('frame.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
crop = autocrop(gray)
cv2.imwrite('framemodThird.png',crop)

True

## Image histogram equalization

In [40]:
from skimage import data, img_as_float
from skimage import exposure
import cv2
import numpy as np

img1 = cv2.imread('histogramNormalization/first.png',0)
img2 = cv2.imread('histogramNormalization/second.png',0)
# Equalization
#img = cv2.imread('wiki.jpg',0)
equ1 = cv2.equalizeHist(img1)
equ2 = cv2.equalizeHist(img2)
res = np.hstack((equ1,equ2)) #stacking images side-by-side
cv2.imwrite('histogramNormalization/res.png',res)


# create a CLAHE object (Arguments are optional).
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
cl1 = clahe.apply(img1)
cl2 = clahe.apply(img2)
res_cl = np.hstack((cl1, cl2)) #stacking images side-by-side
cv2.imwrite('histogramNormalization/res_cl.png',res_cl)



True

# distances between hex values

In [40]:
# This function takes in a list of key value pairs (image key, hex value string) and 
# Finds all combinations of the pairs and then calculated the number of different bits between two hash strings
# It returns a sorted list, sorted in reverse order so the pairs with the highest difference are on the top of the list
def find_hamming_distances(tuple_pair_array):
    distances = []

    # find all combinations
    for i in combinations(tuple_pair_array, 2):
        distance =  get_hamming_distance(i[0][1],i[1][1])
        distances.append((i[0],i[1],distance))

    distances = sorted(distances, key =  lambda x:x[2], reverse=True)

    for distance_pair in distances:
        output = "{}|{} - {}|{} - {}".format(
            distance_pair[0][0],
            distance_pair[1][0],
            distance_pair[0][1],
            distance_pair[1][1],
            distance_pair[2]
        )

        print output


# Functions that finds number of different bits between two hash strings  
def get_hamming_distance(hash_string1, hash_string2):
    """Get the number of different bits between two hash strings."""
    dist = 0
    # get diff matrices from hash string
    bits1 = hash_to_bits(hash_string1)
    bits2 = hash_to_bits(hash_string2)

    # compute distance
    for bit1, bit2 in zip(bits1, bits2):
        if bit1 != bit2:
            dist += 1
    return dist

def hash_to_bits(hash_string):
    """Convert a hash string into the corresponding bit string."""
    bits = []
    # Convert into individual hex numbers
    hex_nums = ['0x' + hash_string[i:i+2] for i in range(0, len(hash_string), 2)]
    for hex_num in hex_nums:
        bit_string = bin(int(hex_num, 16))[2:].rjust(8, '0') # binary string
        bits.append(bit_string)
    return "".join(bits) # return as one string


In [39]:
from itertools import combinations

# key value pair (key, value)

example_hash_strings= [("0WS86GPURFK5","c68686868e0f0e1c"), 
                    ("76KUS3QCGVCY","c78786868e0f0e1c") ,
                    ("96EC4QS20Z28","c78786868e0f0e1c"),
                    ("CL8W7L333U90","c78706868e0f0e1c"),
                    ("FDAZ5NL5NFL2","c7870646ce0f0e1c"),
                    ("HBX8QLI9HH25","c7870686ce0f0e1c"),
                    ("JY2ZAINWD2RX","c68706068e0e0e1c"),
                    ("LP47ZGJ256YU","c78786068e0f0e1e"),
                    ("NTETO8P77N96","c78786868e0f0e1c"),
                    ("SLK2PRXGW3DZ","c78706868e0f0e1c")]

example_hash_strings2= [("0WS86GPURFK5","c68686868e0f0e1c"), 
                    ("76KUS3QCGVCY","c78786868e0f0e1c") ]


find_hamming_distances(example_hash_strings2)


0WS86GPURFK5|76KUS3QCGVCY - c68686868e0f0e1c|c78786868e0f0e1c - 2


In [ ]:
distances

In [ ]:
Result hond 1

--------------------------------------
Video frame length: 311
File name: 0WS86GPURFK5 
c6 86 86 86 8e 0f 0e 1c 
--------------------------------------
Video frame length: 308
File name: 76KUS3QCGVCY 
c7 87 86 86 8e 0f 0e 1c 
--------------------------------------
Video frame length: 259
File name: 96EC4QS20Z28 
c7 87 86 86 8e 0f 0e 1c 
--------------------------------------
Video frame length: 319
File name: CL8W7L333U90 
c7 87 06 86 8e 0f 0e 1c 
--------------------------------------
Video frame length: 402
File name: FDAZ5NL5NFL2 
c7 87 06 46 ce 0f 0e 1c 
--------------------------------------
Video frame length: 332
File name: HBX8QLI9HH25 
c7 87 06 86 ce 0f 0e 1c 
--------------------------------------
Video frame length: 233
File name: JY2ZAINWD2RX 
c6 87 06 06 8e 0e 0e 1c 
--------------------------------------
Video frame length: 419
File name: LP47ZGJ256YU 
c7 87 86 06 8e 0f 0e 1e 
--------------------------------------
Video frame length: 448
File name: NTETO8P77N96 
c7 87 86 86 8e 0f 0e 1c 
--------------------------------------
Video frame length: 282
File name: SLK2PRXGW3DZ 
c7 87 06 86 8e 0f 0e 1c 

# Feature hashing 

In [79]:
from sklearn.feature_extraction import FeatureHasher

hash_string = [
            "f3f3e3e34342c282"
            "e7e7e74747048402",
            "e7e7e7e7e7048402",
            "e7e7e7e3e3000407",
            "f7e7e3e3a3000407",
            "f7f7f3e3e3810004",
            "f7f7f3f3e3818005",
            "e7e7f1f1f3018505",
            "65e7e7e7e7214105",
            "dff6e2e2e203000a",
            "d7e6e6c6c662c00a",
            "f6e6e4e4c426860c",
            "f7e7c4848484c005",
            "f7cfc0c0c0004088",
            "f7c7c1c1c1414181",
            "b7c3c3c3c3828282",
            "674747c7c5850505",
            "c7c38787c7c30303",
            "8383838787878787",
            "43c3470707878f0f",
            "e3c7070707070f0f",
            "733307070787070f",
            "ff130307078707cf",
            "ff878787c7878787",
            "d7c3c3c347474343",
            "e3e3c3c3c3030181",
            "e7c183c3c3838380",
            "f7e7c1c141c1c141",
            "f5e3e1e161c1c000",
            "faf1f1f141416140",
            "fbf0f07041400549",
            "fbf1f0b020602000",
            "f3f0f0e020610000",
            "f9f1f0f0a0e00000",
            "fbbbf0f0d0304000",
            "dffbf8f0f0905001",
            "bffbf8f0d0104000",
            "fdfdf0b08080c002",
            "fdf5f4a484002004",
            "b7b7e4e4c4a40408",
            "f6f6e6c64242040d",
            "e3f3f2e22203020e",
            "f3f3e1c10203020e",
            "f1f1e1c002030a0e",
            "f3f2e2e020020206",
            "f3f2e2e202020206",
            "f3f3f2e020010002",
            "f3f3f1e120018286",
            "fbf3f1f1a181c101",
            "e7fbf1f1f1a18104",
            "f6f3f1f1f1610385",
            "f3f3f1e1e181810c",
            "ebf3f1e0a003020e",
            "eff3f2e26242020a",
            "fff3f2f040420208",
            "f3f3f2e040010082",
            "fbf3f2f240610000",
            "fbfbf2f230110080",
            "fbfbf9f0b0310000",
            "ebf9f3e2a2330206",
            "e3e3e1e223a30246",
            "e1c1e1e223c30206",
            "f9e1e1c001c30202"]

hasher = FeatureHasher(input_type='string', n_features=8)
X = hasher.transform(hash_string)

In [80]:

for i in X.toarray():
    print i

[-1. -5.  0.  2.  5.  0.  0.  9.]
[ 3. -3.  0.  0.  0.  0.  0.  6.]
[ 3. -4.  0.  0.  2.  0.  0.  5.]
[ 2. -4.  1.  1.  3.  0.  0.  3.]
[ 1. -4.  0.  3.  3. -1.  0.  2.]
[ 2. -4.  0.  4.  4. -1.  0.  1.]
[ 2. -3.  0.  3.  3. -3.  0.  2.]
[ 4. -1.  0.  0.  2. -2.  0.  5.]
[ 1. -4.  1.  3.  1.  0.  0.  6.]
[ 6. -5.  1.  1.  0.  0.  0.  3.]
[ 1. -4.  0.  1.  0.  0.  0.  4.]
[-2. -7.  0.  1.  1.  0.  0.  1.]
[  0. -12.   0.   2.   0.   0.   0.   0.]
[ 0. -5.  0.  1.  0. -6.  0.  0.]
[ 1. -7.  0.  0.  4. -1.  0.  3.]
[ 3. -5.  0.  0.  4.  0.  0.  0.]
[ 4. -8.  0.  0.  4.  0.  0.  0.]
[ 5. -8.  0.  0.  3.  0.  0.  0.]
[ 2. -6.  0.  2.  2.  0.  0.  0.]
[ 5. -7.  0.  2.  1.  0.  0.  1.]
[ 6. -6.  0.  1.  3.  0.  0.  0.]
[ 4. -6.  0.  3.  2. -1.  0.  0.]
[ 7. -7.  0.  2.  0.  0.  0.  0.]
[-1. -3.  0.  1.  5.  0.  0.  0.]
[ 0. -6.  0.  0.  6. -2.  0.  2.]
[ 1. -8.  0.  0.  5. -1.  0.  1.]
[ 0. -4.  0.  1.  0. -6.  0.  1.]
[ 1. -5.  0.  1.  2. -4.  0.  3.]
[-2. -1.  1.  4.  0. -6.  0.  0.]
[-2. -

In [85]:
bla = "ebf9f3e2a2330206"

In [97]:
from scipy import spatial

dataSetI = [1, 200, 3, 4]
dataSetII = [1, 2, 3, 4]
result = 1 - spatial.distance.cosine(dataSetI, dataSetII)

print result

0.388756690429


['eb', 'f9', 'f3', 'e2', 'a2', '33', '02', '06']